# sklearn vs tinycudann_sklearn MLPClassifier

This notebook compares:
- `tinycudann_sklearn.MLPClassifier`
- `sklearn.neural_network.MLPClassifier`

For each dataset, both models use the same train/test split, standardization, and matching hyperparameters.
Reported metrics are fit time and test accuracy.

In [2]:
import time
import warnings
from pathlib import Path

import numpy as np
from sklearn.datasets import load_digits, load_iris, make_classification
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier as SklearnMLPClassifier
from sklearn.preprocessing import StandardScaler
from tinycudann_sklearn import MLPClassifier as TinyMLPClassifier


def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "README.md").exists() and (p / "CMakeLists.txt").exists():
            return p
    raise RuntimeError("Could not find tiny-cuda-nn repository root.")


ROOT = find_repo_root(Path.cwd().resolve())
print("Repository root:", ROOT)

warnings.filterwarnings("ignore", category=ConvergenceWarning)
np.random.seed(42)

Repository root: /media/tunguz/3139-3535/tiny-cuda-nn


In [3]:
def run_single_model(model_cls, model_name, X_train, y_train, X_test, y_test, common_kwargs):
    model = model_cls(**common_kwargs)
    t0 = time.perf_counter()
    model.fit(X_train, y_train)
    fit_seconds = time.perf_counter() - t0
    accuracy = float(model.score(X_test, y_test))

    result = {
        "model": model_name,
        "fit_seconds": float(fit_seconds),
        "accuracy": accuracy,
        "n_iter": int(getattr(model, "n_iter_", -1)),
    }

    if model_name == "tinycudann_sklearn":
        result["using_tcnn_backend"] = bool(getattr(model, "_using_tcnn", False))

    return result


def benchmark_dataset(name, X, y, common_kwargs):
    X = np.asarray(X, dtype=np.float32)
    y = np.asarray(y, dtype=np.int64)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42, stratify=y
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train).astype(np.float32)
    X_test = scaler.transform(X_test).astype(np.float32)

    tiny_res = run_single_model(
        TinyMLPClassifier,
        "tinycudann_sklearn",
        X_train,
        y_train,
        X_test,
        y_test,
        common_kwargs,
    )

    sk_res = run_single_model(
        SklearnMLPClassifier,
        "sklearn",
        X_train,
        y_train,
        X_test,
        y_test,
        common_kwargs,
    )

    speedup = sk_res["fit_seconds"] / max(tiny_res["fit_seconds"], 1e-9)

    summary = {
        "dataset": name,
        "n_samples": int(X.shape[0]),
        "n_features": int(X.shape[1]),
        "n_classes": int(len(np.unique(y))),
        "tiny": tiny_res,
        "sklearn": sk_res,
        "speedup_vs_sklearn": float(speedup),
    }
    return summary

In [7]:
iris_X, iris_y = load_iris(return_X_y=True)
digits_X, digits_y = load_digits(return_X_y=True)
synthetic_X, synthetic_y = make_classification(
    n_samples=200_000,
    n_features=128,
    n_informative=64,
    n_redundant=8,
    n_classes=10,
    class_sep=1.5,
    flip_y=0.01,
    random_state=42,
)

experiments = [
    (
        "iris",
        iris_X,
        iris_y,
        {
            "hidden_layer_sizes": (64, 64),
            "max_iter": 80,
            "batch_size": 32,
            "learning_rate_init": 5e-3,
            "random_state": 42,
            "early_stopping": True,
            "n_iter_no_change": 8,
            "validation_fraction": 0.1,
            "verbose": False,
        },
    ),
    (
        "digits",
        digits_X,
        digits_y,
        {
            "hidden_layer_sizes": (64, 64),
            "max_iter": 80,
            "batch_size": 128,
            "learning_rate_init": 5e-3,
            "random_state": 42,
            "early_stopping": True,
            "n_iter_no_change": 8,
            "validation_fraction": 0.1,
            "verbose": False,
        },
    ),
    (
        "synthetic_200k",
        synthetic_X,
        synthetic_y,
        {
            "hidden_layer_sizes": (128, 64),
            "max_iter": 30,
            "batch_size": 1024,
            "learning_rate_init": 3e-3,
            "random_state": 42,
            "early_stopping": True,
            "n_iter_no_change": 8,
            "validation_fraction": 0.1,
            "verbose": False,
        },
    ),
]

results = []
for name, X, y, kwargs in experiments:
    results.append(benchmark_dataset(name, X, y, kwargs))

for r in results:
    print()
    print(f"Dataset: {r['dataset']}")
    print(f"  size: samples={r['n_samples']} features={r['n_features']} classes={r['n_classes']}")
    print(
        "  tinycudann_sklearn: "
        f"time={r['tiny']['fit_seconds']:.3f}s "
        f"acc={r['tiny']['accuracy']:.4f} "
        f"iters={r['tiny']['n_iter']} "
        f"backend={r['tiny'].get('using_tcnn_backend')}"
    )
    print(
        "  sklearn:            "
        f"time={r['sklearn']['fit_seconds']:.3f}s "
        f"acc={r['sklearn']['accuracy']:.4f} "
        f"iters={r['sklearn']['n_iter']}"
    )
    print(f"  speedup (sklearn/tiny): {r['speedup_vs_sklearn']:.2f}x")



Dataset: iris
  size: samples=150 features=4 classes=3
  tinycudann_sklearn: time=0.302s acc=0.9474 iters=57 backend=True
  sklearn:            time=0.028s acc=0.9211 iters=18
  speedup (sklearn/tiny): 0.09x

Dataset: digits
  size: samples=1797 features=64 classes=10
  tinycudann_sklearn: time=0.245s acc=0.9667 iters=20 backend=True
  sklearn:            time=0.071s acc=0.9689 iters=14
  speedup (sklearn/tiny): 0.29x

Dataset: synthetic_200k
  size: samples=200000 features=128 classes=10
  tinycudann_sklearn: time=3.236s acc=0.9514 iters=19 backend=False
  sklearn:            time=11.317s acc=0.9474 iters=25
  speedup (sklearn/tiny): 3.50x


In [5]:
# Basic checks to ensure larger-than-Iris datasets are present and results are sane.
iris_n = next(r[1].shape[0] for r in experiments if r[0] == "iris")
for name, X, _, _ in experiments:
    if name != "iris":
        assert X.shape[0] > iris_n, f"{name} is not larger than iris"

res_by_name = {r["dataset"]: r for r in results}
assert res_by_name["iris"]["tiny"]["accuracy"] >= 0.80
assert res_by_name["digits"]["tiny"]["accuracy"] >= 0.90
assert res_by_name["synthetic_20k"]["tiny"]["accuracy"] >= 0.75

print("All checks passed.")

All checks passed.
